## Fine-Tuning on Cheat Environment

In [1]:
from train import HangmanTrainer
from env import HangmanCheat
from network import Network, CheatAgent

In [2]:
# create the cheat environment
cheat_env = HangmanCheat(word_src='words/top_1000.txt')

In [3]:
player = CheatAgent(Network(cheat_env.maxlen), cheat_env.maxlen)
player.model.summary()

Model: "fullmodel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ state_input         │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_embedding     │ (None, 13, 100)   │      3,000 │ state_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 13)        │          0 │ state_input[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_bilstm_1      │ (None, 13, 200)   │    160,800 │ state_embedding[… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_bilstm_2      │ (None, 13, 200)   │    240,800 │ state_bilstm_1[0… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_input       │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_pooling       │ (None, 200)       │          0 │ state_bilstm_2[0… │
│ (GlobalAveragePool… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_dense_1     │ (None, 60)        │      1,620 │ guessed_input[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_dense (Dense) │ (None, 100)       │     20,100 │ state_pooling[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_dense_2     │ (None, 60)        │      3,660 │ guessed_dense_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ state_dense[0][0… │
│ (Concatenate)       │                   │            │ guessed_dense_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │     16,100 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 26)        │      2,626 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 448,706 (1.71 MB)

 Trainable params: 448,706 (1.71 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# load pre-trained weights
player.load_pretrained_agent('models/policy.h5')
player.model.summary()
#player.load_weights('models/finetuned/policy_finetuned_2000.h5')

Model: "fullmodel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ state_input         │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_embedding     │ (None, 13, 100)   │      3,000 │ state_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 13)        │          0 │ state_input[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_bilstm_1      │ (None, 13, 200)   │    160,800 │ state_embedding[… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_bilstm_2      │ (None, 13, 200)   │    240,800 │ state_bilstm_1[0… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_input       │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_pooling       │ (None, 200)       │          0 │ state_bilstm_2[0… │
│ (GlobalAveragePool… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_dense_1     │ (None, 60)        │      1,620 │ guessed_input[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_dense (Dense) │ (None, 100)       │     20,100 │ state_pooling[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_dense_2     │ (None, 60)        │      3,660 │ guessed_dense_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ state_dense[0][0… │
│ (Concatenate)       │                   │            │ guessed_dense_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │     16,100 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ new_output (Dense)  │ (None, 27)        │      2,727 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 448,807 (1.71 MB)

 Trainable params: 448,807 (1.71 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# create the trainer
trainer = HangmanTrainer(player=player, word_src='words/top_1000.txt', save_path='models/finetuned/policy_finetuned_test')

Max length of words is 13


Model: "fullmodel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ state_input         │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_embedding     │ (None, 13, 100)   │      3,000 │ state_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 13)        │          0 │ state_input[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_bilstm_1      │ (None, 13, 200)   │    160,800 │ state_embedding[… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_bilstm_2      │ (None, 13, 200)   │    240,800 │ state_bilstm_1[0… │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_input       │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_pooling       │ (None, 200)       │          0 │ state_bilstm_2[0… │
│ (GlobalAveragePool… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_dense_1     │ (None, 60)        │      1,620 │ guessed_input[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_dense (Dense) │ (None, 100)       │     20,100 │ state_pooling[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_dense_2     │ (None, 60)        │      3,660 │ guessed_dense_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ state_dense[0][0… │
│ (Concatenate)       │                   │            │ guessed_dense_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │     16,100 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ new_output (Dense)  │ (None, 27)        │      2,727 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 448,807 (1.71 MB)

 Trainable params: 448,807 (1.71 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# save player weights
player.model.save(f"models/finetuned/policy_finetuned_0.h5", include_optimizer=False)

In [6]:
trainer.fine_tune(cheat_env, n_trials=100)

Loss : 3.853              :   4%|▍         | 4/100 [00:13<04:26,  2.77s/it]

Loss : 3.738              :   5%|▌         | 5/100 [00:15<03:56,  2.49s/it]

Loss : 3.621              :  12%|█▏        | 12/100 [00:38<04:36,  3.14s/it]

17
17
17
17
17
request
[[27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27. 27. 27. 27. 27. 27. 27.  0.  0.  0.  0.  0.  0.]
 [27.  5. 27. 27.  5. 27. 27.  0.  0.  0.  0.  0.  0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 

Loss : 3.590              :  13%|█▎        | 13/100 [00:45<05:04,  3.50s/it]


KeyboardInterrupt: 